
# Project 2: Armageddon - Atmospheric entry and disruption of asteroids

[Gareth Collins](http://www.imperial.ac.uk/people/g.collins)


## Synopsis:

Asteroids entering Earth’s atmosphere are subject to extreme drag forces that decelerate, heat and disrupt the space rocks. The fate of an asteroid is a complex function of its initial mass, speed, trajectory angle and internal strength. 

[Asteroids](https://en.wikipedia.org/wiki/Asteroid) 10-100 m in diameter can penetrate deep into Earth’s atmosphere and disrupt catastrophically, generating an atmospheric disturbance ([airburst](https://en.wikipedia.org/wiki/Air_burst)) that can cause [damage on the ground](https://www.youtube.com/watch?v=tq02C_3FvFo). Such an event occurred over the city of [Chelyabinsk](https://en.wikipedia.org/wiki/Chelyabinsk_meteor) in Russia, in 2013, releasing energy equivalent to about 520 [kilotons of TNT](https://en.wikipedia.org/wiki/TNT_equivalent) (1 kt TNT is equivalent to $4.184 \times 10^{12}$ J), and injuring thousands of people ([Popova et al., 2013](http://doi.org/10.1126/science.1242642); [Brown et al., 2013](http://doi.org/10.1038/nature12741)). An even larger event occurred over [Tunguska](https://en.wikipedia.org/wiki/Tunguska_event), an unpopulated area in Siberia, in 1908. 

<img src="images/chelyabinsk.png" width="640">

The purpose of this exercise is to develop a fast numerical simulator to predict the fate of asteroids entering Earth’s atmosphere for the purposes of hazard assessment.


## Problem definition

### Basic equations

The dynamics of an asteroid in Earth’s atmosphere is governed by a coupled set of ordinary differential equations:


<table width="600">
    <tbody>
        <tr style="background-color: #FFFFFF;">
            <td><img src="images/variables.png" width="220"> </td>
            <td>
\begin{aligned} 
\frac{dv}{dt} & = \frac{-C_D\rho_a A v^2}{2 m} + g \sin \theta \\
\frac{dm}{dt} & = \frac{-C_H\rho_a A v^3}{2 Q} \\
\frac{d\theta}{dt} & = \frac{g\cos\theta}{v} - \frac{C_L\rho_a A v}{2 m} - \frac{v\cos\theta}{R_P + z} \\
\frac{dz}{dt} & = -v\sin\theta \\
\frac{dx}{dt} & = \frac{v\cos\theta}{1 + z/R_P}
\end{aligned}
            </td>
        </tr>
    </tbody>
</table>

In these equations, $v$, $m$, and $A$ are the asteroid (aka meteoroid) speed (along trajectory), mass and cross-sectional area, respectively. $\theta$ is the meteoroid trajectory angle to the horizontal (in radians), $x$ is the downrange distance of the meteoroid from its entry position, $z$ is the altitude and $t$ is time; $C_D$ is the drag coefficient, $\rho_a$ is the atmospheric density (a function of altitude ), $C_H$ is an ablation efficiency coefficient, $Q$ is the specific heat of ablation; $C_L$ is a lift coefficient; and $R_P$ is the planetary radius. All terms use MKS units.


### Asteroid break-up and spreading

It is common to assume (for simplicity) that, prior to break-up, the radius (and cross-sectional area) of the asteroid remains constant; that is, any mass-loss by **ablation** does not change the cross-sectional area of the asteroid. We will further assume a spherical asteroid. 

A commonly used criterion for the break-up of an asteroid in the atmosphere is when the ram pressure of the air interacting with the asteroid $\rho_a v^2$ first exceeds the strength of the asteroid $Y$.

$$\rho_a v^2 = Y$$

Should break-up occur, the asteroid deforms and spreads laterally as it continues its passage through the atmosphere. As a result its radius and cross-sectional area increase, but the asteroid density is assumed to remain constant (often referred to as 'pancaking'). It is conventional to define the cross-sectional area of the expanding asteroid as $A = \pi r^2$ (i.e., assuming a circular cross-section), for use in the above equations. 

Several models for the radial spreading rate $\frac{dr}{dt}$ have been proposed. In the simplest model, the fragmented asteroid’s spreading rate is related to its along trajectory speed [(Hills and Goda, 1993)](http://doi.org/10.1086/116499):

$$ \frac{dr}{dt} = \left[\frac{7}{2}\alpha\frac{\rho_a}{\rho_m}\right]^{1/2} v$$

Where $r$ is the asteroid radius, $\rho_m$ is the asteroid density (assumed constant) and $\alpha$ is a spreading coefficient, often taken to be 0.3. 

Note that fragmentation and spreading **ceases** ($\frac{dr}{dt} = 0$) when the ram pressure drops back below the strength of the asteroid $\rho_a v^2 < Y$.


### A typical solution

The figure below shows a typical solution to the above set of equations for an impact airburst scenario. The kinetic energy loss per unit height, which is a good approximation for the energy transferred to the atmosphere as heat (and is often referred to as the energy deposited per km altitude), is shown as a function of altitude. 

<img src="images/airburst.png" width="640">

In this scenario the asteroid experiences breakup at approximately 48-km altitude (denoted by the star), before spreading and decelerating rapidly until an altitude of approximately 15 km at which point the **energy loss per unit height** is **maximum**. This point is often considered to be the **burst altitude**. The total kinetic energy lost by the asteroid at this point is a good estimate of the airburst energy (i.e., the total energy deposited into the atmosphere) for hazard analysis.

Impact scenarios with a burst altitude well (>5 km) above the surface can be considered to be **airburst events** and are unlikely to form a sizable crater on the ground. In such cases, even if a substantial fraction of the original asteroid survives to the ground it will be decelerated to a very small fraction of its initial speed.

Impact scenarios with a burst altitude below the surface (i.e., peak energy deposition per unit height is not reached before the asteroid strikes the ground) will form a sizable impact crater on the ground and can be considered as a **cratering event** (although a sizable proportion of the original kinetic energy of the asteroid may be transferred to the air).

Between these scenarios is a complex regime where a **low altitude airburst combined with a substantial crater-forming event** is likely. This regime is not well understood.


## Challenge

Your task is to develop a Python program to solve this system of equations. In the following, we describe the functionality that we would like you to incorporate into your program, in increasing order of complexity and decreasing order of reward. 

### Core functionality

Write a function that takes the following inputs

* Asteroid radius (m)
* Asteroid speed (m/s)
* Asteroid density (kg/m$^3$)
* Asteroid strength (Pa)
* Asteroid trajectory angle (degrees)
* Solution **output** timestep (s)

and returns a Pandas dataframe with the following columns:

* Time (s), altitude (m), horizontal position (m), speed (m/s), mass (kg), radius (m), kinetic energy loss per unit height (kt/km)

and a dictionary of outcomes:

* Whether the scenario is an airburst or a cratering event
* The peak kinetic energy loss per unit height in kt per km (airburst)
* The burst altitude in m (airburst)
* The total kinetic energy loss at burst in kt (airburst)
* The mass (kg) and speed (m/s) of the remnant of the asteroid that strikes the ground (cratering)
* The time (s) that the remnant of the asteroid that strikes the ground (cratering)

As you will see from the code-stubs that you are provided with in your repository, we have defined a class `Planet` that includes all coefficients in the model. By default these should be set to the values given in Table 1 below. Please retain this structure to your code as access to this class is required for the scoring system. 

To achieve the core functionality, you can assume that atmospheric density is defined by a simple exponential relationship: 

$$
\rho_a = \rho_0 e^{-z/H}, 
$$

where $\rho_0$ is the air density at the ground, $z$ is altitude and $H$ is the atmospheric scale height.

You should develop a simple interface for a User to interact with your tool (e.g., jupyter notebook, command-line arguments or example python scripts) and documented functionality to produce simple graphical output of the evolution of the asteroid in the atmosphere (e.g., plots of mass, speed, energy, etc., as a function of altitude or time).

Note that the initial alitude $z_0$ must be an optional input parameter to your tool, with a recommended value of 100 km. 

Note that the time step of the **output** must be an input parameter to your tool; however, the numerical integration time step and/or solver tolerance is up to you as you seek to find the best compromise between solution accuracy and expediency. Your results dataframe must return your simulation output at the User-specified time interval.


You can assume the following value for coefficients

| Symbol |   Description    | Units/Value |
|:------:|:----------------:|:-----------:|
| $C_D$  |    Drag coefficient  |     1      |
| $C_H$  | Heat Transfer coef.   |     0.1     |
| $Q$    | Heat of ablation (J/kg) |   $10^7$    |
| $C_L$  |    Lift coefficient    |  $10^{-3}$  |
| $\alpha$ | Dispersion coefficient | 0.3 |
| $\rho_0$ | Air density at zero altitude (kg/m$^3$) | 1.2 |
| $H$  | Atmospheric scale height (m) |  8000  | 
| $R_P$ | Planet radius (Earth; m) | 6371,000 |
| $x_0$ | Initial $x$-position (m) | 0. |
| $z_0$ | Initial $z$-position (m) | 100,000. |



### Extension Functionality

Additional credit will be given if your software demonstrates the following optional extended capabilities:

* Ability to use a tabulated atmospheric density profile instead of an exponential atmosphere (a tabulated atmospheric profile for Earth is provided in [here](data/AltitudeDensityTable.csv)).

* Ability to apply the model to Mars using the following equation of state for Mars' atmosphere:
\begin{aligned} 
p & = 0.699 \exp(-0.00009 z) \\
T & = 249.7 - 0.00222 z , \, \text{for} \, z \geq 7000 \\
T & = 242.1 - 0.000998 z , \, \text{for} \, z \lt 7000 \\
\rho & = \frac{p}{0.1921T} \\
\end{aligned}
Note that in these equations $\rho$ is the atmospheric density in kg m$^{-3}$, $p$ is the pressure in kPa, $T$ is the temperature in K and $z$ is the altitude in m.

* Ability to determine asteroid parameters (e.g., strength and radius) that best fit an observed energy deposition curve. As an example, you can use the energy deposition curve inferred for the Chelyabinsk event (available [here](data/ChelyabinskEnergyAltitude.csv)). Note that in this data file energy is given in units of [kilotons of TNT](https://en.wikipedia.org/wiki/TNT_equivalent), where 1 kt TNT is equivalent to $4.184 \times 10^{12}$ J. Note also that the density 3300 kg/m$^3$, impact angle (18.3 degrees to horizontal) and entry velocity (19.2 km/s) are well known from observations ([Popova et al., 2013](http://doi.org/10.1126/science.1242642)). 

* Ability to perform an ensemble of simulations with **probability distributions as inputs** and **return the distribution of burst altitude**. The cumulative probability distribution to use for each input parameter is:
    * Radius: $$P(>r) = \frac{r-r_{min}}{r_{max}-r_{min}} ,$$
where $r_{min}$ and $r_{max}$ are User-specified.
    * Angle: $$P(>\theta) = \cos^2\theta .$$
    * Strength: $$P(>Y) = \frac{\log{(Y/Y_{min})}}{\log{(Y_{max}/Y_{min})}} ,$$
where $Y_{min} = 1$ kPa and $Y_{max} = 10$ MPa.
    * Velocity (at infinity): $$P(>v) = \text{erf}\left(\frac{v}{a\sqrt{2}}\right)-\frac{v}{a}\exp\left\{\frac{-v^2}{2 a^2}\right\}\sqrt{\frac{2}{\pi}} ,$$ where $a = 11$ km s$^{-1}$. Note that impact velocity is then computed from $$v_i = \sqrt{v_{esc}^2 + v^2},$$ where $v_{esc}$ is the target planet's escape velocity ($v_{esc} = 11.2$ km s$^{-1}$ for Earth).
    * Density: $$P(>\rho) = \frac{1}{2}\left[1 + \text{erf} \left( \frac{\rho-\rho_m}{\sigma_\rho\sqrt{2}} \right) \right] ,$$
where $\rho_m = 3000$ kg m$^{-3}$ $\sigma_\rho = 1000$ kg m$^{-3}$.

## Assessment

The **deadline** for software submission is **Friday 13th December, 12:00 noon**.

### Software (70 marks)

Your software will be assessed primarily on functionality (**30/70 marks**), performance (**20/70 marks**) and sustainability (**20/70 marks**).

Functionality (**30 marks**): Your software will be scored based on its ability to perform a number of automated tests. These are:

1. A comparison between your software output and an analytical solution (**5 marks**). The analytical solution is derived with the simplifying assumptions of exponential atmosphere ($\rho_a = \rho_0 e^{-z/H}$), no gravitational acceleration ($g=0$), a flat planet ($R_p = \infty$), no lift ($C_L = 0$), no mass change owing to ablation ($\frac{dm}{dt}=0$) and no fragmentation ($\sigma_0 = \infty$, $\frac{dr}{dt}=0$). Note that you should derive this analytical solution and incorporate this into your test framework. An initial altitude of $z_0 = 100,000$ m will be used for this test.

2. Correct solution for six sets of asteroid parameters (**15 marks**). Quality of the solution will be based on the error in burst altitude and peak energy deposition rate, as well as solution variables as a function of time. An initial altitude of $z_0 = 100,000$ m will be used for each of these tests.  The following six scenarios will be tested:
    * Four impacts on Earth with different impactor parameters (from within a realistic distribution) and assuming an exponential atmosphere.
    * The same impact on Earth as one of the above scenarios, using the tabulated terrestrial atmosphere provided.
    * The same impact as another of the above scenarios but on Mars, using the equation of state for Mars' atmosphere provided.
    

3. The distribution of burst altitudes obtained given distributions of input parameters (**10 marks**). 
    * Radius
    * Angle
    * Strength
    * Velocity
    * Density

This family of tests will include five tests with a distribution in each individual parameter (above) and another four tests where multiple parameter distributions are included at the same time, with each test including an additional parameter in the order above (i.e., the first multi-parameter test will be radius and angle; the last will be all of the variables). The fiducial scenario for this exercise will be: $r=10$ m; $\theta = 45^\circ$; $Y = 10^5$ Pa; $v_i = 21,000$ m s$^{-1}$; $\rho = 3000$ kg m$^{-3}$. Parameters not varied will take these values. The shape of the distribution should be set as per these instructions, but the type of sampling and the number of samples employed is up to you. You are free to define your own initial altitude for the ensemble analysis to improve efficiency if you wish.
    
Performance (**20 marks**): The time of completion of each of the tests in parts 2 and 3 above will be used to measure the performance of your tool, both to calculate a single scenario and to calculate and synthesise an ensemble of outcomes.

Sustainability (**20 marks**): As with all software projects, you should employ all the elements of best practice in software development that you have learned so far. A GitHub repository will be created for your project to host your software. The quality and sustainability of your software and its documentation will be assessed based on your final repository and how it evolves during the week. Specific attention will be given to the following elements:

1. Installation and usage instructions
2. Documentation (in HTML / PDF format). Note that a template SPHINX documentation framework has been provided for you as a starting point for your documentation.
3. Coding style
4. Quality and coverage of automatic testing framework
5. General repository usage
6. Licensing

Please refer to the ACSE-4 handbook for more information about the assessment of software quality.

### Presentation (20 marks)

On Friday afternoon, you must demonstrate your software to staff during a 90 minute interactive session. Each assessor will ask you to demonstrate one of the following five elements of your software. Up to four marks will be awarded for each element assessed. Note that elements (1)-(3) are considered part of the core functionality; elements (4) and (5) are extention features that will be assessed if you have completed them.

1. A brief description of your solution algorithm, including ODE solving routine.
2. A brief demonstration of how a User would interact with your software and the graphical output provided by your software.
3. A quantification of the accuracy of your numerical solution for two cases, with and without fragmentation, for User-specified input parameters. It is up to you to design an appropriate demonstration of accuracy, but this could take the form of a plot of error vs. timestep size or error vs. solver tolerance, depending on your solution algorithm. 
4. A demonstration of how to use your software to determine the impactor parameters (asteroid size & strength) for the Chelyabinsk airburst, by fitting your model solutions of kinetic energy loss per unit height vs. altitude to the inferred energy deposition curve.
5. A demonstration of functionality to perform a statistical ensemble; that is, to take probability distributions of each parameter as inputs and return outcomes as probability distributions.


### Teamwork (peer assessment; 10 marks)

After the presentations, you will complete a self-evaluation of your group's performance. This will inform the teamwork component of your mark. Please refer to the ACSE-4 guidelines for more information about the assessment of teamwork.

## Technical requirements
* You should use the assigned GitHub repository exclusively for your project
* Your software must be written in Python 3.7
* You can import anything from the standard python 3.7 libraries as well as numpy, matplotlib, scipy, mpltools and sympy
* For solving the ODEs you **must write your own ODE solution algorithm**. You can make use of any of your own implementations of algorithms that you have learned in ACSE 3, however you are not permitted to use the in-built ODE solvers in scipy (e.g., `odeint`, `solve_ivp`).
* You are not allowed to import any other python packages (if in doubt, please query with the Module Coordinator)
* You can assume that Users of your software will have pytest installed, so this does not need to be part of your repository
* You should use Travis and the GitHub flow for any automated testing that you implement
* You do not need to make a Graphical User Interface for your software: the program can be run in the command line or in a Python 3.7 environment
